This file is to put together all the data and create a test set. (and maybe evaluation set as well)

In [2]:
import csv
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import pickle

In [13]:
badwords = ['nigger', 'fuck', 'sex', 'negro', 'cunt', 'bitch', 'cum', 'gay', 'dyke', 'fag', 'faggot']
redditwords = ['Edit:', 'edit:']


def check_bad_word(badwordlist, sentence):
    sentencecopy = sentence.split(' ')
    for word in badwords:
        if word in sentencecopy:
            return False
    return True

Read pickle files and label (preparing the new dataset)

In [4]:
filename = "humorous_jokes_new_raw.pickle"
new_pos_data = pickle.load(open(filename,'rb'))
new_neg_data = pickle.load(open("negatives_new_raw.pickle",'rb'))

In [7]:
pos_df = pd.DataFrame(new_pos_data,columns=["text"])
neg_df = pd.DataFrame(new_neg_data,columns=["text"])

In [11]:
pos_df['label'] = 1
neg_df['label'] = 0

In [12]:
neg_df.head()

,text,label
0,A Bad Beginning Makes A Bad Ending,0
1,A Bad Corn Promise Is Better Than A Good Lawsuit,0
2,A Bad Workman Quarrels With His Tools,0
3,A Bargain Is A Bargain,0
4,A Beggar Can Never Be Bankrupt,0


In [17]:
passed_indeces = [check_bad_word(badwords, sent) for sent in pos_df['text']]
pos_filt = pos_df[passed_indeces]

In [21]:
passed_indeces = [check_bad_word(badwords, sent) for sent in neg_df['text']]
neg_filt = neg_df[passed_indeces]

read in previous dataset and label them

In [24]:
qa_jokes = pd.read_csv("qa_total.csv")
non_qa_jokes = pd.read_csv("non_qa_total.csv")

In [28]:
qa_jokes['text'] = qa_jokes['title'] +" "+ qa_jokes['selftext']
qa_jokes=qa_jokes.drop(['title','selftext','score','source'],axis=1)

In [32]:
non_qa_jokes['text'] = non_qa_jokes['title'] +" "+ non_qa_jokes['selftext']
non_qa_jokes=non_qa_jokes.drop(['title','selftext','score','source'],axis=1)

In [34]:
all_jokes = qa_jokes.append(non_qa_jokes, ignore_index=True)
all_jokes['label'] = 1

put together all the positive data

In [38]:
all_jokes = all_jokes.append(pos_df,ignore_index=True)

Load more negative examples

In [88]:
neg_ex = pd.read_csv("~/Workspace/pal-model/humor_challenge_data/raw/results-20200305-223056.csv")

In [90]:
temp_Df = neg_ex.copy()
temp_Df = temp_Df.drop(['selftext'],axis=1)
temp_Df = temp_Df.rename(columns={"body":"text"})
temp_Df = temp_Df[temp_Df['text'] != "[removed]"  ]
temp_Df = temp_Df[temp_Df['text'] != "[deleted]"  ]

In [91]:
neg_ex=neg_ex.drop(['body'],axis=1)
neg_ex = neg_ex.rename(columns={"selftext" : "text"}).append(temp_Df,ignore_index=True)

In [92]:
neg_ex = neg_ex.append(temp_Df,ignore_index=True)

In [94]:
neg_ex['label']=0

In [95]:
all_negatives = neg_ex.append(neg_df,ignore_index=True)

combine all the data and divide into test and train dataset

In [97]:
all_examples = all_jokes.append(all_negatives,ignore_index=True)

In [100]:
msk = np.random.rand(len(all_examples)) < 0.8
train = all_examples[msk]
test = all_examples[~msk]


In [106]:
def write_csv(posts, name):
    name = name + ".csv"
    posts.to_csv(name, index=False)

In [107]:
write_csv(all_negatives,"all_negatives")
write_csv(all_jokes,"all_jokes")
write_csv(train,"train_data")
write_csv(test,"test_data")
write_csv(all_examples,"all_examples")

Randomly shuffle train and test data